Thus far, the graphs which have been built to represent conversational dynamics, provide a ststic struture which lacks the ability to capture the sequential flow of conversation, and consequently its importance in identifying conversational cohesion. To address this shortcoming, we will employ Temporal Graph Neural Networks to represent the flow of conversation. Several TGNNs exist, but they all address a similar task; the interpretation of graph structures that change over time. 

For graphs G1, G2, ..., Gn, a TGNN has the following structure G1 -> G2 -> G3 -> ... -> Gn. Wheras most TGNN tasks focus on node, link, or graph prediciton, we will focus on sequence classification, to define the cohesion of a meeting based on its progression. We will perform seuqeence level classification using pytorch geometric pooling methods. 

The static graphs produced until now, cover the paralinguistic aspects of conversation (speaking rate, interruptions, language complexity, etc.), as the importance of these features is less dependent on their progression through tie. Our temporal analysis will predominantly concern itself with the flow of information in a meeting, as the reception and expression of inforation by each group meeber is largley dependent on the progression of conversation. 

Before delving into the impleentation of graphs, lets first define the assumed conversational scenario, as the preservation of this dynamic throughout analyzed segments being analyzed is crucial for the developed sturctures to be grounded in reality. Most importantly, all meetings considered must be such that each participant's turn is always: 

    1. The only current turn 
    2. Meant to be received (albeit to different degrees) by all participants
    3. Heard and understood by all participants 

All this said, the conversational structures developed are such that:
 1. Each Graph, Gn, within the sequence represent the expression of information by one node towards other nodes (one participant turn).
 2. Each Graph, Gn, will have n - 1 directed edges, where n is the number of nodes, all stemming from the node currently speaking towards all other nodes. 
 3. The value of each directed edge A -> B will be: 
        A. An emedding representing the information being expressed by A
        B. Weighted in reference to the number of turns since B spoke 
 4. Each Node A will contain an aggregate of embeddings received from turn taking nodes, as well as expressed by itself, with embeddings based on the basis of recency. 

This graph structure will allow for participants to be defined over the course of a meeting in reference to their involvement in the exchange of information. 